In [17]:
#Import Dependencies
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import calendar
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

In [18]:
#Set Preferences
myoptions = webdriver.ChromeOptions()
prefs = {
  "translate_whitelists": {"ja":"en"},
  "translate":{"enabled":"true"}
}
myoptions.add_experimental_option("prefs", prefs)
d = webdriver.Chrome('C:\Windows/chromedriver', options=myoptions)

#Define URL
url = "http://www15.plala.or.jp/gcap/disney/"
d.get(url)
d.maximize_window()

In [23]:
#Create Lists to Place Scraped Data
months = []
for monthnum in range(3):

    #Scrape Calendar Data
    elem = d.find_element_by_class_name("FXTABLE") 
    data = elem.text

    i = 1
    new_data = []
    temp = []
    for e in (data.split("\n")[14:]):
        temp.append(e)
        if i%4==0:
            new_data.append(temp)
            temp = []
        i = i+1
    #Place Scraped Data into DataFrame
    months.append(pd.DataFrame(new_data, columns=["Date", "Crowd Size", "Temperature", "Opening Hours"]))
    #Go to Previous Month
    element = WebDriverWait(d, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input'))
    )
    previous = d.find_element_by_xpath('//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input')
    ActionChains(d).click(previous).perform()
    element = WebDriverWait(d, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="hyo"]/table/tbody/tr[2]/td[1]/font/font/input'))
    )
print(months)

[      Date Crowd Size Temperature Opening Hours
0       31         16      11 | 2          9-19
1      2/1         16      12 | 2          9-19
2        2         16      15 | 6          9-19
3        3         16      10 | 4          9-19
4     Four         16      11 | 3          9-19
5     Five         16      11 | 4          9-19
6        6         16      14 | 4          9-19
7        7         16      15 | 6          9-19
8        8         16       9 | 4          9-19
9        9         16       9 | 2          9-19
10     Ten         16         Ten       12 | -2
11    9-19         11          16            20
12  14 | 1       9-19          12            16
13      20     13 | 2        9-19            13
14      16         30      14 | 6         10-19
15      14         16          40        16 | 6
16   10-19         15          16            60
17  14 | 6      10-19          16            16
18      30     11 | 4       10-19            17
19      16     16 | 9       10-19      

In [24]:
#Place Scraped Data into DataFrame
disneyland_scrape = pd.concat(months)

In [25]:
#Save DataFrame as CSV
disneyland_scrape.to_csv(r'C:\Users\Mickey\Desktop\Final_Project\disneyland_scrape.csv', index = False)